In [1]:
import statistics
import queue
from collections import namedtuple

In [2]:
Item = namedtuple("Item", ['index', 'value', 'weight', 'value_per_weight'])
items = [Item(index=0, value=1, weight=2, value_per_weight=0.5), Item(index=1, value=1, weight=2, value_per_weight=0.5), Item(index=2, value=1, weight=2, value_per_weight=0.5), Item(index=3, value=13, weight=7, value_per_weight=1.8571428571428572), Item(index=4, value=10, weight=5, value_per_weight=2.0), Item(index=5, value=10, weight=5, value_per_weight=2.0), Item(index=6, value=8, weight=3, value_per_weight=2.6666666666666665)]

In [3]:
# Item = namedtuple("Item", ['index', 'value', 'weight'])
# items = [Item(index=0, value=45, weight=5), Item(index=1, value=48, weight=8), Item(index=2, value=35, weight=3)]

In [4]:
Node = namedtuple("Node", ['direction', 'index', 'branch', 'value', 'weight', 'estimate'])

In [5]:
def get_children(node_value, capacity, node_estimate, items, branch):

    if branch > len(items):
        return None, None
        
    item = items[branch-1]
    
    if capacity-item.weight >= 0 :
        left_node = Node('left', item.index, branch, node_value+item.value, capacity-item.weight, node_estimate)
        right_node = Node('right', item.index, branch, node_value, capacity, node_estimate-item.value)
    else:
        left_node = Node('left', item.index, branch, -1, capacity-item.weight, -1)
        right_node = Node('right', item.index, branch, node_value, capacity, node_estimate-item.value)

    return left_node, right_node, branch

## Best First Search

In [413]:
unexplored_nodes = []
max_value = 0
max_capacity = 10
total_value = sum([item.value for item in items])
total_items = len(items)
iterations = 0

In [414]:
left_node, right_node, current_branch = get_children(0, max_capacity, total_value, items, 1)
unexplored_nodes.append(left_node)
unexplored_nodes.append(right_node)
unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.estimate)
unexplored_nodes

[Node(direction='left', index=0, branch=1, value=-1, weight=-89991, estimate=-1),
 Node(direction='right', index=0, branch=1, value=0, weight=10, estimate=1482855)]

In [412]:
explore_node = unexplored_nodes.pop()

left_node, right_node, current_branch = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch+1)

if current_branch != total_items:
    
    if left_node.value != left_node.estimate:
        unexplored_nodes.append(left_node)
        
    if right_node.value != right_node.estimate:
        unexplored_nodes.append(right_node)
else:
    if left_node.value > right_node.value and left_node.value > max_value:
        max_value = left_node.value
        
    elif right_node.value > left_node.value and right_node.value > max_value:
        max_value = right_node.value

filtered_nodes = [node for node in unexplored_nodes if node.value > max_value]    

unexplored_nodes = sorted(filtered_nodes, key=lambda x: x.estimate)

iterations += 1
print(max_value)
if not unexplored_nodes:
    print(iterations)
unexplored_nodes

0


[Node(direction='right', index=12, branch=25, value=0, weight=10, estimate=52515)]

## Depth First Search

In [1017]:
unexplored_nodes = []
max_value = 0


# If the estimate is lower than minimum available value; dont branch further. The available value is the best value
max_capacity = 100000
min_capacity = min([item.weight for item in items])

total_value = 99998.88859177056
total_items = len(items)
iterations = 0

In [1018]:
min_capacity

10002

In [1019]:
# min(item.weight for item in items)
items[:5]

[Item(index=0, value=90000, weight=90001, value_per_weight=0.9999888890123443),
 Item(index=1, value=89750, weight=89751, value_per_weight=0.9999888580628629),
 Item(index=2, value=10001, weight=10002, value_per_weight=0.9999000199960008),
 Item(index=3, value=89500, weight=89501, value_per_weight=0.9999888269404811),
 Item(index=4, value=10252, weight=10254, value_per_weight=0.9998049541642285)]

In [1020]:
left_node, right_node, current_branch = get_children(0, max_capacity, total_value, items, 1)
unexplored_nodes.append(right_node)
unexplored_nodes.append(left_node)
# unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.branch, x.estimate))
unexplored_nodes

[Node(direction='right', index=0, branch=1, value=0, weight=100000, estimate=9998.888591770563),
 Node(direction='left', index=0, branch=1, value=90000, weight=9999, estimate=99998.88859177056)]

In [1021]:
explore_node = unexplored_nodes.pop()

# while True:
#     if explore_node.branch+1 > total_items or explore_node.estimate < max_value:
#         max_value = explore_node.value if explore_node.value > max_value else max_value
#         if explore_node:
#             explore_node = unexplored_nodes.pop()
#     else:
#         break

optimistic_value + current_value <= max_value:

left_node, right_node, current_branch = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch+1)


print('Exploring: ', explore_node)
print('Max Value: ', max_value)
print('Left Child: ', left_node)
print('Right Child: ', right_node)


if (left_node.weight >= min_capacity and current_branch != total_items) and left_node.value+left_node.estimate >= max_value:
    unexplored_nodes.append(left_node)
else:     
    max_value = left_node.value if left_node.value > max_value else max_value
    
if (right_node.weight >= min_capacity and current_branch != total_items) and right_node.value+right_node.estimate >= max_value:
    unexplored_nodes.append(right_node)   
else:     
    max_value = right_node.value if right_node.value > max_value else max_value

print('Max Value: ', max_value)
print(unexplored_nodes)
print('---------------------------------------')

Exploring:  Node(direction='left', index=0, branch=1, value=90000, weight=9999, estimate=99998.88859177056)
Max Value:  0
Left Child:  Node(direction='left', index=1, branch=2, value=-1, weight=-79752, estimate=-1)
Right Child:  Node(direction='right', index=1, branch=2, value=90000, weight=9999, estimate=10248.888591770563)
Max Value:  90000
[Node(direction='right', index=0, branch=1, value=0, weight=100000, estimate=9998.888591770563)]
---------------------------------------


In [688]:
# Exploring:  Node(direction='left', index=3, branch=1, value=13, weight=2, estimate=21)
# Max Value:  0
# Left Child:  Node(direction='left', index=4, branch=2, value=-1, weight=-3, estimate=-1)
# Right Child:  Node(direction='right', index=4, branch=2, value=13, weight=2, estimate=11)
# Max Value:  0
# [Node(direction='right', index=3, branch=1, value=0, weight=10, estimate=8), Node(direction='right', index=4, branch=2, value=13, weight=2, estimate=11)]
# ---------------------------------------

In [386]:
explore_node = unexplored_nodes.pop()

left_node, right_node, current_branch = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch+1)
print(left_node)
print(right_node)


if current_branch != total_items:
    if left_node.weight >= 0:
        unexplored_nodes.append(left_node)
        unexplored_nodes.append(right_node)
    else:
        unexplored_nodes.append(right_node)
else:
    max_value = left_node.value if left_node.value > max_value else max_value
    
# unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.branch, x.estimate))

print(max_value)
unexplored_nodes

Node(direction='left', index=28, branch=17, value=-1, weight=-3029, estimate=-1)
Node(direction='right', index=28, branch=17, value=89750, weight=10249, estimate=-1233515.1114082295)
90000


[Node(direction='right', index=1, branch=2, value=0, weight=100000, estimate=-79751.11140822944),
 Node(direction='right', index=28, branch=17, value=89750, weight=10249, estimate=-1233515.1114082295)]

In [ ]:
[Node(direction='left', index=1, branch=2, value=89750, weight=10249, estimate=9999.0)]

In [ ]:
right_node

In [ ]:
round(2.6666, 2)

In [1076]:
with open('./data/ks_50_0', 'r') as input_data_file:
    input_data = input_data_file.read()

In [1077]:
# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []

for i in range(1, item_count+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1]),  int(parts[0])/int(parts[1])))

In [1078]:
total_value = 0
total_weight = capacity
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight > 0 :
        total_value += item.value
        total_weight -= item.weight  
    else:
        total_value += total_weight * item.value_per_weight
        break

In [1079]:
items = sorted(items, key=lambda x: x.weight, reverse=True)
total_value

142418.67875227213

In [1080]:
def optimistic_estimate(items, current_index, current_weight, remaining_capacity):
    optimistic_value = 0
    while current_index < len(items) and current_weight + items[current_index].weight <= remaining_capacity:
        optimistic_value += items[current_index].value
        remaining_capacity -= items[current_index].weight
        current_index += 1
    if current_index < len(items):
        optimistic_value += remaining_capacity * items[current_index].value_per_weight
    return optimistic_value

def knapsack_dfs_bb(items, max_weight, current_index, current_weight, current_value):
    if current_weight > max_weight:
        return 0
    
    if current_index >= len(items):
        return current_value
    
    optimistic_value = optimistic_estimate(items, current_index, current_weight, max_weight - current_weight)
    if optimistic_value + current_value <= max_value:
        return 0
    
    value_with_item = knapsack_dfs_bb(items, max_weight, current_index + 1,
                                       current_weight + items[current_index].weight,
                                       current_value + items[current_index].value)
    
    value_without_item = knapsack_dfs_bb(items, max_weight, current_index + 1,
                                          current_weight, current_value)
    
    return max(value_with_item, value_without_item)

In [1081]:
knapsack_dfs_bb(items, capacity, 0, 0, 0)

KeyboardInterrupt: 